In [65]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
#All values read as string

data1 = pd.read_csv("data_0deg_025mps.csv")
data2 = pd.read_csv("data_0deg_05mps.csv")
data3 = pd.read_csv("data_0deg_075mps.csv")
data4 = pd.read_csv("data_0deg_1mps.csv")

data5 = pd.read_csv("data_5deg_025mps.csv")
data6 = pd.read_csv("data_5deg_05mps.csv")
data7 = pd.read_csv("data_5deg_075mps.csv")
data8 = pd.read_csv("data_5deg_1mps.csv")

print(data1.shape)

data1 = data1[0:2000]
data2 = data2[0:2000]
data3 = data3[0:2000]
data4 = data4[0:2000]

data5 = data5[0:2000]
data6 = data6[0:2000]
data7 = data7[0:2000]
data8 = data8[0:2000]
print(data1.shape)

(6192, 7)
(2000, 7)


In [3]:
tempData = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8])
num = tempData.reset_index().values
print(num.shape)

(16000, 8)


In [4]:
tempData.head()

,X [ m ] (X Coordinate),Y [ m ] (Y Coordinate),Z [ m ] (Z Coordinate),Pressure [ Pa ],Velocity u [ m s^-1 ],Velocity v [ m s^-1 ],Velocity w [ m s^-1 ]
0,0.01184,0.05000,-0.00400,-5.22373,-0.00430,-0.13310,-0.00176
1,0.01080,0.05059,-0.00396,-5.66039,0.00309,-0.00654,-0.00142
2,0.01189,0.05107,-0.00385,-5.34105,-0.00128,-0.11913,-0.00291
3,-0.01189,0.04892,-0.00385,-4.77085,0.00172,-0.11952,0.00022
4,-0.01184,0.05000,-0.00400,-5.00739,-0.00120,-0.10649,-0.00097


In [5]:
Vel025 = np.full((2000,1),0.25)
Vel05 = np.full((2000,1),0.5)
Vel075 = np.full((2000,1),0.75)
Vel1 = np.full((2000,1),1)

Angle0 = np.full((8000,1),0)
Angle5 = np.full((8000,1),5)

Vel = np.concatenate((Vel025,Vel05,Vel075,Vel1,Vel025,Vel05,Vel075,Vel1), axis = 0)

#Vel = np.insert(Vel,0,0,axis=0)

Angle = np.concatenate((Angle0,Angle5), axis = 0)

#Angle = np.insert(Angle,0,0,axis=0)

print(Vel.shape)
print(Angle.shape)

(16000, 1)
(16000, 1)


In [10]:
print(num.shape)

dataVel = np.concatenate((num,Vel), axis = 1)
dataAng = np.concatenate((dataVel,Angle), axis = 1)

dataArray = dataAng

print(dataArray.shape)

data2 = np.insert(dataArray,0,['Number','X_Coordinate','Y_Coordinate','Z_Coordinate','Pressure','U_Velocity','V_Velocity','W_Velocity','Vel','Angle'],axis=0)

print(data2)

finalData = pd.DataFrame(data=data2[1:,1:], index=data2[1:,0], columns=data2[0,1:])  
print(finalData.shape)

data = finalData

columnsTitles=['X_Coordinate','Y_Coordinate','Z_Coordinate','U_Velocity','V_Velocity','W_Velocity','Vel','Angle','Pressure']
data=data.reindex(columns=columnsTitles)

(16000, 8)
(16000, 10)
[['Number' 'X_Coordinate' 'Y_Coordinate' ... 'W_Velocity' 'Vel' 'Angle']
 [0 '0.01184' 0.05 ... -0.00176 0.25 0]
 [1 '0.01080' 0.050589999999999996 ... -0.0014199999999999998 0.25 0]
 ...
 [1997 '0.00965' 0.0325 ... 0.014130000000000002 1.0 5]
 [1998 '0.00367' 0.028460000000000003 ... -0.03758 1.0 5]
 [1999 '0.00024' 0.021580000000000002 ... -0.05565 1.0 5]]
(16000, 9)


In [15]:
data = data.values
print(data)

[['0.01184' 0.05 -0.004 ... 0.25 0 -5.223730000000001]
 ['0.01080' 0.050589999999999996 -0.00396 ... 0.25 0 -5.6603900000000005]
 ['0.01189' 0.05107 -0.00385 ... 0.25 0 -5.34105]
 ...
 ['0.00965' 0.0325 -0.0083 ... 1.0 5 -23.002660000000002]
 ['0.00367' 0.028460000000000003 -0.01268 ... 1.0 5 -20.71848]
 ['0.00024' 0.021580000000000002 0.01126 ... 1.0 5 -31.50752]]


In [17]:
print(data.shape)

(16000, 9)


In [26]:
Y = data[:,8]
Y.shape

(16000,)

In [28]:
X = data[:,0:8]
X.shape

(16000, 8)

In [58]:
# define model
## Add batch normalization
### Normalize inputs beforehand?
#### Check leaky_relu for improved performance
##### Cross Validate other loss functions
def first_model():
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=8, kernel_initializer='normal', activation='relu')) #dim 8 or 7?
    model.add(Dense(64, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [59]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator1 = KerasRegressor(build_fn=first_model, epochs=100, batch_size=5, verbose=1)

In [61]:
kfold = KFold(n_splits=10, random_state=seed) 
results1 = cross_val_score(estimator1, X, Y, cv=kfold) #Test model using 10 k-fold cross-entropy
print("Results: %.2f (%.2f) MSE" % (results1.mean(), results.std()))

Epoch 1/100
14400/14400 [==============================] - 7s 476us/step - loss: 1480.7074
Epoch 2/100
14400/14400 [==============================] - 5s 372us/step - loss: 1276.3745
Epoch 3/100
14400/14400 [==============================] - 6s 409us/step - loss: 1126.3426
Epoch 4/100
14400/14400 [==============================] - 6s 386us/step - loss: 848.8878
Epoch 5/100
14400/14400 [==============================] - 6s 385us/step - loss: 733.1011
Epoch 6/100
14400/14400 [==============================] - 6s 428us/step - loss: 686.7935
Epoch 7/100
14400/14400 [==============================] - 6s 403us/step - loss: 653.8799
Epoch 8/100
14400/14400 [==============================] - 6s 398us/step - loss: 610.6183
Epoch 9/100
14400/14400 [==============================] - 8s 533us/step - loss: 583.7536
Epoch 10/100
14400/14400 [==============================] - 6s 420us/step - loss: 563.1491
Epoch 11/100
14400/14400 [==============================] - 7s 499us/step - loss: 535.5307
Epoch

14400/14400 [==============================] - 7s 459us/step - loss: 89.4395
Epoch 91/100
14400/14400 [==============================] - 6s 421us/step - loss: 55.3700
Epoch 92/100
14400/14400 [==============================] - 7s 462us/step - loss: 66.9161
Epoch 93/100
14400/14400 [==============================] - 8s 550us/step - loss: 53.5141
Epoch 94/100
14400/14400 [==============================] - 7s 504us/step - loss: 55.8204
Epoch 95/100
14400/14400 [==============================] - 8s 575us/step - loss: 64.9331
Epoch 96/100
14400/14400 [==============================] - 9s 629us/step - loss: 68.8427
Epoch 97/100
14400/14400 [==============================] - 8s 540us/step - loss: 93.0310
Epoch 98/100
14400/14400 [==============================] - 8s 580us/step - loss: 52.1562
Epoch 99/100
14400/14400 [==============================] - 9s 599us/step - loss: 53.0749
Epoch 100/100
1600/1600 [==============================] - 1s 622us/step
Epoch 1/100
14400/14400 [===============

14400/14400 [==============================] - 9s 592us/step - loss: 53.1657
Epoch 80/100
14400/14400 [==============================] - 9s 628us/step - loss: 77.6471
Epoch 81/100
14400/14400 [==============================] - 8s 588us/step - loss: 52.0609
Epoch 82/100
14400/14400 [==============================] - 9s 629us/step - loss: 53.2985
Epoch 83/100
14400/14400 [==============================] - 9s 608us/step - loss: 61.8182
Epoch 84/100
14400/14400 [==============================] - 9s 617us/step - loss: 58.5812
Epoch 85/100
14400/14400 [==============================] - 9s 623us/step - loss: 52.6589
Epoch 86/100
14400/14400 [==============================] - 9s 595us/step - loss: 65.6780
Epoch 87/100
14400/14400 [==============================] - 9s 641us/step - loss: 47.0222
Epoch 88/100
14400/14400 [==============================] - 8s 567us/step - loss: 49.6746
Epoch 89/100
14400/14400 [==============================] - 9s 610us/step - loss: 57.6439
Epoch 90/100
14400/1440

14400/14400 [==============================] - 9s 598us/step - loss: 67.6906
Epoch 69/100
14400/14400 [==============================] - 9s 620us/step - loss: 78.7277
Epoch 70/100
14400/14400 [==============================] - 8s 579us/step - loss: 77.4418
Epoch 71/100
14400/14400 [==============================] - 9s 627us/step - loss: 71.3890
Epoch 72/100
14400/14400 [==============================] - 9s 628us/step - loss: 73.5635
Epoch 73/100
14400/14400 [==============================] - 9s 622us/step - loss: 62.5934
Epoch 74/100
14400/14400 [==============================] - 10s 720us/step - loss: 102.0362
Epoch 75/100
14400/14400 [==============================] - 8s 551us/step - loss: 56.7230
Epoch 76/100
14400/14400 [==============================] - 8s 536us/step - loss: 81.9984
Epoch 77/100
14400/14400 [==============================] - 8s 539us/step - loss: 71.4442
Epoch 78/100
14400/14400 [==============================] - 7s 486us/step - loss: 60.2389
Epoch 79/100
14400/14

14400/14400 [==============================] - 9s 593us/step - loss: 122.2271
Epoch 58/100
14400/14400 [==============================] - 9s 616us/step - loss: 154.9973
Epoch 59/100
14400/14400 [==============================] - 8s 587us/step - loss: 104.8422
Epoch 60/100
14400/14400 [==============================] - 9s 618us/step - loss: 129.9041
Epoch 61/100
14400/14400 [==============================] - 9s 612us/step - loss: 105.5728
Epoch 62/100
14400/14400 [==============================] - 9s 625us/step - loss: 100.4561
Epoch 63/100
14400/14400 [==============================] - 9s 652us/step - loss: 99.5094
Epoch 64/100
14400/14400 [==============================] - 8s 574us/step - loss: 105.3247
Epoch 65/100
14400/14400 [==============================] - 9s 617us/step - loss: 129.5768
Epoch 66/100
14400/14400 [==============================] - 8s 575us/step - loss: 88.8105
Epoch 67/100
14400/14400 [==============================] - 9s 659us/step - loss: 87.3046
Epoch 68/100
14

...

14400/14400 [==============================] - 6s 386us/step - loss: 94.0688
Epoch 81/100
14400/14400 [==============================] - 6s 417us/step - loss: 76.2634
Epoch 82/100
14400/14400 [==============================] - 6s 382us/step - loss: 88.6045
Epoch 83/100
14400/14400 [==============================] - 6s 386us/step - loss: 77.0080
Epoch 84/100
14400/14400 [==============================] - 6s 416us/step - loss: 72.1451
Epoch 85/100
14400/14400 [==============================] - 6s 387us/step - loss: 75.1692
Epoch 86/100
14400/14400 [==============================] - 6s 390us/step - loss: 78.6246
Epoch 87/100
14400/14400 [==============================] - 6s 417us/step - loss: 90.7750
Epoch 88/100
14400/14400 [==============================] - 6s 389us/step - loss: 75.6773
Epoch 89/100
14400/14400 [==============================] - 6s 389us/step - loss: 75.0714
Epoch 90/100
14400/14400 [==============================] - 6s 414us/step - loss: 88.8853
Epoch 91/100
14400/1440

14400/14400 [==============================] - 9s 615us/step - loss: 61.3393
Epoch 70/100
14400/14400 [==============================] - 9s 592us/step - loss: 71.9393
Epoch 71/100
14400/14400 [==============================] - 9s 606us/step - loss: 61.8958
Epoch 72/100
14400/14400 [==============================] - 11s 757us/step - loss: 62.6624
Epoch 73/100
14400/14400 [==============================] - 9s 656us/step - loss: 61.7864
Epoch 74/100
14400/14400 [==============================] - 9s 644us/step - loss: 62.1782
Epoch 75/100
14400/14400 [==============================] - 9s 608us/step - loss: 69.0078
Epoch 76/100
14400/14400 [==============================] - 9s 626us/step - loss: 61.5103
Epoch 77/100
14400/14400 [==============================] - 9s 646us/step - loss: 80.3653
Epoch 78/100
14400/14400 [==============================] - 9s 598us/step - loss: 52.5499
Epoch 79/100
14400/14400 [==============================] - 9s 637us/step - loss: 62.2568
Epoch 80/100
14400/144

14400/14400 [==============================] - 10s 662us/step - loss: 79.3951
Epoch 59/100
14400/14400 [==============================] - 9s 624us/step - loss: 79.6932
Epoch 60/100
14400/14400 [==============================] - 9s 656us/step - loss: 76.7427
Epoch 61/100
14400/14400 [==============================] - 9s 652us/step - loss: 73.6361
Epoch 62/100
14400/14400 [==============================] - 9s 647us/step - loss: 74.2281
Epoch 63/100
14400/14400 [==============================] - 9s 653us/step - loss: 76.6185
Epoch 64/100
14400/14400 [==============================] - 9s 614us/step - loss: 66.8042
Epoch 65/100
14400/14400 [==============================] - 10s 669us/step - loss: 69.0573
Epoch 66/100
14400/14400 [==============================] - 9s 642us/step - loss: 72.0220
Epoch 67/100
14400/14400 [==============================] - 10s 673us/step - loss: 68.8906
Epoch 68/100
14400/14400 [==============================] - 10s 667us/step - loss: 75.8754
Epoch 69/100
14400/

14400/14400 [==============================] - 9s 653us/step - loss: 88.5877
Epoch 48/100
14400/14400 [==============================] - 9s 640us/step - loss: 81.7318
Epoch 49/100
14400/14400 [==============================] - 10s 692us/step - loss: 114.7673
Epoch 50/100
14400/14400 [==============================] - 10s 703us/step - loss: 71.3636
Epoch 51/100
14400/14400 [==============================] - 9s 634us/step - loss: 91.3208
Epoch 52/100
14400/14400 [==============================] - 10s 669us/step - loss: 94.2098
Epoch 53/100
14400/14400 [==============================] - 9s 633us/step - loss: 80.70820s
Epoch 54/100
14400/14400 [==============================] - 9s 656us/step - loss: 72.6234
Epoch 55/100
14400/14400 [==============================] - 10s 665us/step - loss: 82.7453
Epoch 56/100
14400/14400 [==============================] - 9s 636us/step - loss: 73.5169
Epoch 57/100
14400/14400 [==============================] - 9s 659us/step - loss: 79.0473
Epoch 58/100
144

14400/14400 [==============================] - 16s 1ms/step - loss: 78.9504
Epoch 38/100
14400/14400 [==============================] - 17s 1ms/step - loss: 67.3803
Epoch 39/100
14400/14400 [==============================] - 16s 1ms/step - loss: 79.2137
Epoch 40/100
14400/14400 [==============================] - 16s 1ms/step - loss: 67.0028
Epoch 41/100
14400/14400 [==============================] - 17s 1ms/step - loss: 67.6588
Epoch 42/100
14400/14400 [==============================] - 17s 1ms/step - loss: 61.2337
Epoch 43/100
14400/14400 [==============================] - 17s 1ms/step - loss: 64.2223
Epoch 44/100
14400/14400 [==============================] - 16s 1ms/step - loss: 67.0266
Epoch 45/100
14400/14400 [==============================] - 17s 1ms/step - loss: 63.6736 0s - l -
Epoch 46/100
14400/14400 [==============================] - 17s 1ms/step - loss: 58.6842
Epoch 47/100
14400/14400 [==============================] - 20s 1ms/step - loss: 57.4660
Epoch 48/100
14400/14400 

In [71]:
model = Sequential()
model.add(Dense(128, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(64, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_percentage_error', 'MSE'])
# Fit the model
model.fit(X, Y, epochs=100, batch_size=5, verbose=1, validation_split=0.1)
# evaluate the model
scores = model.evaluate(X, Y, verbose=1)
print("\n\n %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

## Validation looks like model may have overfit?? Probably need more data / less params || Could fix with dropout?
## Check memorization by randomizing labels

Train on 14400 samples, validate on 1600 samples
Epoch 1/100
14400/14400 [==============================] - 10s 670us/step - loss: 942.0640 - mean_absolute_percentage_error: 91.6848 - mean_squared_error: 942.0640 - val_loss: 4296.9437 - val_mean_absolute_percentage_error: 63.2133 - val_mean_squared_error: 4296.9437
Epoch 2/100
14400/14400 [==============================] - 7s 519us/step - loss: 772.4878 - mean_absolute_percentage_error: 59.6149 - mean_squared_error: 772.4878 - val_loss: 4205.5782 - val_mean_absolute_percentage_error: 77.0417 - val_mean_squared_error: 4205.5782
Epoch 3/100
14400/14400 [==============================] - 8s 550us/step - loss: 634.6518 - mean_absolute_percentage_error: 53.9228 - mean_squared_error: 634.6518 - val_loss: 3395.3012 - val_mean_absolute_percentage_error: 68.1504 - val_mean_squared_error: 3395.3012
Epoch 4/100
14400/14400 [==============================] - 8s 522us/step - loss: 490.6138 - mean_absolute_percentage_error: 50.0288 - mean_squared_er

14400/14400 [==============================] - 7s 492us/step - loss: 115.1875 - mean_absolute_percentage_error: 33.0523 - mean_squared_error: 115.1875 - val_loss: 1294.5204 - val_mean_absolute_percentage_error: 38.2816 - val_mean_squared_error: 1294.5204
Epoch 32/100
14400/14400 [==============================] - 6s 451us/step - loss: 118.7566 - mean_absolute_percentage_error: 34.0550 - mean_squared_error: 118.7566 - val_loss: 1424.6235 - val_mean_absolute_percentage_error: 52.4418 - val_mean_squared_error: 1424.6235
Epoch 33/100
14400/14400 [==============================] - 7s 507us/step - loss: 114.5052 - mean_absolute_percentage_error: 32.8616 - mean_squared_error: 114.5052 - val_loss: 1168.0307 - val_mean_absolute_percentage_error: 44.5723 - val_mean_squared_error: 1168.0307
Epoch 34/100
14400/14400 [==============================] - 7s 463us/step - loss: 105.9500 - mean_absolute_percentage_error: 32.1474 - mean_squared_error: 105.9500 - val_loss: 1124.9190 - val_mean_absolute_per

Epoch 62/100
14400/14400 [==============================] - 6s 447us/step - loss: 44.7375 - mean_absolute_percentage_error: 27.5498 - mean_squared_error: 44.7375 - val_loss: 788.4719 - val_mean_absolute_percentage_error: 34.9302 - val_mean_squared_error: 788.4719
Epoch 63/100
14400/14400 [==============================] - 10s 697us/step - loss: 51.8929 - mean_absolute_percentage_error: 24.9547 - mean_squared_error: 51.8929 - val_loss: 768.6297 - val_mean_absolute_percentage_error: 36.3267 - val_mean_squared_error: 768.6297
Epoch 64/100
14400/14400 [==============================] - 11s 776us/step - loss: 45.3494 - mean_absolute_percentage_error: 27.8905 - mean_squared_error: 45.3494 - val_loss: 743.9541 - val_mean_absolute_percentage_error: 35.2515 - val_mean_squared_error: 743.9541
Epoch 65/100
14400/14400 [==============================] - 9s 601us/step - loss: 48.3959 - mean_absolute_percentage_error: 27.3050 - mean_squared_error: 48.3959 - val_loss: 818.7904 - val_mean_absolute_per

14400/14400 [==============================] - 6s 445us/step - loss: 34.6229 - mean_absolute_percentage_error: 21.7977 - mean_squared_error: 34.6229 - val_loss: 690.7705 - val_mean_absolute_percentage_error: 41.5373 - val_mean_squared_error: 690.7705
Epoch 94/100
14400/14400 [==============================] - 7s 491us/step - loss: 33.6578 - mean_absolute_percentage_error: 24.8370 - mean_squared_error: 33.6578 - val_loss: 606.9389 - val_mean_absolute_percentage_error: 36.0909 - val_mean_squared_error: 606.9389
Epoch 95/100
14400/14400 [==============================] - 6s 440us/step - loss: 30.9710 - mean_absolute_percentage_error: 22.6922 - mean_squared_error: 30.9710 - val_loss: 596.2915 - val_mean_absolute_percentage_error: 31.6721 - val_mean_squared_error: 596.2915
Epoch 96/100
14400/14400 [==============================] - 7s 465us/step - loss: 30.3977 - mean_absolute_percentage_error: 22.6081 - mean_squared_error: 30.3977 - val_loss: 751.7555 - val_mean_absolute_percentage_error: 

In [72]:
from keras.models import load_model

model.save('model1.h5')  # creates a HDF5 file 'my_model.h5'

#model = load_model('model1.h5')

import dill
dill.dump_session('notebook_env1.db') # should save environment?
#dill.load_session('notebook_env1.db')


ModuleNotFoundError: No module named 'dill'

In [73]:
m = load_model('model1.h5')

In [98]:
print(y_test[0])
print(X_test[0])

#test = np.array([0.045, 0.00015, 0.107, -0.0135, 0.0395, 1.0, 5])
#print(test.shape)
#print(model.predict(test,batch_size=None, verbose=0, steps=None))

predictions = model.predict(X)
print(Y[0])
print(predictions[0])

-92.47802
[0.04609 0.00014 0.10722999999999999 -0.013619999999999998 0.03912 1.0 5]
-5.223730000000001
[-5.1907845]


In [38]:
# define model low accuracy
#def two_model():
#    # create model
#    model = Sequential()
#    model.add(Dense(64, input_dim=8, kernel_initializer='normal', activation='sigmoid'))
#    model.add(Dense(32, kernel_initializer='normal', activation='sigmoid'))
#    model.add(Dense(8, kernel_initializer='normal', activation='sigmoid'))
#    model.add(Dense(1, kernel_initializer='normal'))
#    # Compile model
#    model.compile(loss='mean_squared_error', optimizer='adam')
#    return model

In [39]:
#estimator = KerasRegressor(build_fn=two_model, epochs=100, batch_size=5, verbose=1)

In [40]:
#kfold = KFold(n_splits=10, random_state=seed)
#results = cross_val_score(estimator, X, Y, cv=kfold)
#print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
14400/14400 [==============================] - 4s 246us/step - loss: 2562.2417
Epoch 2/100
14400/14400 [==============================] - 3s 199us/step - loss: 2041.0284
Epoch 3/100
14400/14400 [==============================] - 3s 196us/step - loss: 1946.6559
Epoch 4/100
14400/14400 [==============================] - 4s 263us/step - loss: 1936.0455
Epoch 5/100
14400/14400 [==============================] - 3s 230us/step - loss: 1904.1710
Epoch 6/100
14400/14400 [==============================] - 3s 230us/step - loss: 1728.7102
Epoch 7/100
14400/14400 [==============================] - 4s 256us/step - loss: 1705.4877
Epoch 8/100
14400/14400 [==============================] - 3s 240us/step - loss: 1701.5332
Epoch 9/100
14400/14400 [==============================] - 3s 209us/step - loss: 1699.4041
Epoch 10/100
14400/14400 [==============================] - 3s 199us/step - loss: 1697.8273
Epoch 11/100
14400/14400 [==============================] - 3s 197us/step - loss: 1695.61

14400/14400 [==============================] - 3s 234us/step - loss: 1673.3094
Epoch 91/100
14400/14400 [==============================] - 3s 242us/step - loss: 1672.3953
Epoch 92/100
14400/14400 [==============================] - 3s 236us/step - loss: 1672.4349
Epoch 93/100
14400/14400 [==============================] - 4s 264us/step - loss: 1673.8425
Epoch 94/100
14400/14400 [==============================] - 3s 222us/step - loss: 1672.6151
Epoch 95/100
14400/14400 [==============================] - 3s 215us/step - loss: 1672.1104
Epoch 96/100
14400/14400 [==============================] - 4s 253us/step - loss: 1671.7545
Epoch 97/100
14400/14400 [==============================] - 4s 271us/step - loss: 1672.4763
Epoch 98/100
14400/14400 [==============================] - 4s 278us/step - loss: 1672.3489
Epoch 99/100
14400/14400 [==============================] - 3s 222us/step - loss: 1671.5269
Epoch 100/100
1600/1600 [==============================] - 0s 233us/step
Epoch 1/100
14400/14

14400/14400 [==============================] - 3s 223us/step - loss: 674.4594
Epoch 79/100
14400/14400 [==============================] - 3s 227us/step - loss: 674.9145
Epoch 80/100
14400/14400 [==============================] - 3s 229us/step - loss: 670.7796
Epoch 81/100
14400/14400 [==============================] - 3s 215us/step - loss: 667.0001
Epoch 82/100
14400/14400 [==============================] - 3s 204us/step - loss: 665.9167
Epoch 83/100
14400/14400 [==============================] - 3s 195us/step - loss: 659.5456
Epoch 84/100
14400/14400 [==============================] - 3s 226us/step - loss: 658.3256
Epoch 85/100
14400/14400 [==============================] - 4s 255us/step - loss: 656.8796
Epoch 86/100
14400/14400 [==============================] - 4s 243us/step - loss: 652.8886
Epoch 87/100
14400/14400 [==============================] - 3s 187us/step - loss: 647.3398
Epoch 88/100
14400/14400 [==============================] - 3s 195us/step - loss: 644.7720
Epoch 89/100

KeyboardInterrupt: 

In [41]:
# define model
def three_model():
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(64, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(32, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [42]:
estimator = KerasRegressor(build_fn=three_model, epochs=100, batch_size=5, verbose=1)

In [43]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
14400/14400 [==============================] - 4s 300us/step - loss: 2551.0326
Epoch 2/100
14400/14400 [==============================] - 3s 227us/step - loss: 2041.3716
Epoch 3/100
14400/14400 [==============================] - 3s 235us/step - loss: 1775.4890
Epoch 4/100
14400/14400 [==============================] - 4s 249us/step - loss: 1501.8372
Epoch 5/100
14400/14400 [==============================] - 4s 269us/step - loss: 1352.5680
Epoch 6/100
14400/14400 [==============================] - 3s 227us/step - loss: 1189.0788
Epoch 7/100
14400/14400 [==============================] - 4s 252us/step - loss: 1082.7337
Epoch 8/100
14400/14400 [==============================] - 4s 248us/step - loss: 1005.6830
Epoch 9/100
14400/14400 [==============================] - 4s 268us/step - loss: 957.4861
Epoch 10/100
14400/14400 [==============================] - 3s 233us/step - loss: 903.3761
Epoch 11/100
14400/14400 [==============================] - 4s 252us/step - loss: 891.9337


14400/14400 [==============================] - 3s 212us/step - loss: 245.8361
Epoch 92/100
14400/14400 [==============================] - 4s 302us/step - loss: 247.6947
Epoch 93/100
14400/14400 [==============================] - 4s 245us/step - loss: 243.2858
Epoch 94/100
14400/14400 [==============================] - 3s 228us/step - loss: 228.0071
Epoch 95/100
14400/14400 [==============================] - 3s 229us/step - loss: 234.5063
Epoch 96/100
14400/14400 [==============================] - 4s 266us/step - loss: 254.6513
Epoch 97/100
14400/14400 [==============================] - 4s 251us/step - loss: 219.2123
Epoch 98/100
14400/14400 [==============================] - 4s 245us/step - loss: 262.0819
Epoch 99/100
14400/14400 [==============================] - 4s 244us/step - loss: 227.1259
Epoch 100/100
1600/1600 [==============================] - 1s 355us/step
Epoch 1/100
14400/14400 [==============================] - 5s 321us/step - loss: 2541.2729
Epoch 2/100
14400/14400 [=====

14400/14400 [==============================] - 3s 216us/step - loss: 212.7836
Epoch 81/100
14400/14400 [==============================] - 4s 250us/step - loss: 182.0087
Epoch 82/100
14400/14400 [==============================] - 3s 215us/step - loss: 187.2859
Epoch 83/100
14400/14400 [==============================] - 3s 216us/step - loss: 185.1489
Epoch 84/100
14400/14400 [==============================] - 3s 215us/step - loss: 180.9859
Epoch 85/100
14400/14400 [==============================] - 3s 218us/step - loss: 180.8019
Epoch 86/100
14400/14400 [==============================] - 5s 322us/step - loss: 189.5261
Epoch 87/100
14400/14400 [==============================] - 3s 239us/step - loss: 192.6728
Epoch 88/100
14400/14400 [==============================] - 4s 245us/step - loss: 162.1626
Epoch 89/100
14400/14400 [==============================] - 3s 241us/step - loss: 190.1977
Epoch 90/100
14400/14400 [==============================] - 5s 319us/step - loss: 177.8270
Epoch 91/100

14400/14400 [==============================] - 4s 254us/step - loss: 164.8028
Epoch 70/100
14400/14400 [==============================] - 3s 222us/step - loss: 164.9717
Epoch 71/100
14400/14400 [==============================] - 3s 220us/step - loss: 181.4407
Epoch 72/100
14400/14400 [==============================] - 3s 220us/step - loss: 159.2821
Epoch 73/100
14400/14400 [==============================] - 3s 222us/step - loss: 156.8876
Epoch 74/100
14400/14400 [==============================] - 4s 255us/step - loss: 153.8836
Epoch 75/100
14400/14400 [==============================] - 4s 284us/step - loss: 165.2760
Epoch 76/100
14400/14400 [==============================] - 4s 297us/step - loss: 151.2783
Epoch 77/100
14400/14400 [==============================] - 4s 265us/step - loss: 177.4173
Epoch 78/100
14400/14400 [==============================] - 4s 305us/step - loss: 157.4231
Epoch 79/100
14400/14400 [==============================] - 4s 292us/step - loss: 171.0562
Epoch 80/100

14400/14400 [==============================] - 4s 262us/step - loss: 1620.2890
Epoch 58/100
14400/14400 [==============================] - 3s 226us/step - loss: 1619.9296
Epoch 59/100
14400/14400 [==============================] - 3s 229us/step - loss: 1620.2966
Epoch 60/100
14400/14400 [==============================] - 3s 225us/step - loss: 1619.5358
Epoch 61/100
14400/14400 [==============================] - 3s 228us/step - loss: 1619.7089
Epoch 62/100
14400/14400 [==============================] - 4s 262us/step - loss: 1619.8587
Epoch 63/100
14400/14400 [==============================] - 4s 244us/step - loss: 1620.1686
Epoch 64/100
14400/14400 [==============================] - 4s 288us/step - loss: 1620.5239
Epoch 65/100
14400/14400 [==============================] - 4s 258us/step - loss: 1619.3119
Epoch 66/100
14400/14400 [==============================] - 4s 295us/step - loss: 1619.8826
Epoch 67/100
14400/14400 [==============================] - 4s 258us/step - loss: 1620.7407
E

...

14400/14400 [==============================] - 4s 284us/step - loss: 1623.4893
Epoch 22/100
14400/14400 [==============================] - 4s 296us/step - loss: 1619.3649
Epoch 23/100
14400/14400 [==============================] - 4s 257us/step - loss: 1614.2149
Epoch 24/100
14400/14400 [==============================] - 4s 258us/step - loss: 1611.9350
Epoch 25/100
14400/14400 [==============================] - 4s 261us/step - loss: 1610.5843
Epoch 26/100
14400/14400 [==============================] - 5s 321us/step - loss: 1605.8536
Epoch 27/100
14400/14400 [==============================] - 4s 269us/step - loss: 1616.1499
Epoch 28/100
14400/14400 [==============================] - 4s 274us/step - loss: 1607.0023
Epoch 29/100
14400/14400 [==============================] - 4s 271us/step - loss: 1609.9755
Epoch 30/100
14400/14400 [==============================] - 5s 320us/step - loss: 1608.9105
Epoch 31/100
14400/14400 [==============================] - 4s 274us/step - loss: 1611.0081
E

14400/14400 [==============================] - 4s 287us/step - loss: 1200.7320
Epoch 10/100
14400/14400 [==============================] - 4s 272us/step - loss: 1124.7752
Epoch 11/100
14400/14400 [==============================] - 4s 304us/step - loss: 1049.2414
Epoch 12/100
14400/14400 [==============================] - 4s 287us/step - loss: 981.1911
Epoch 13/100
14400/14400 [==============================] - 4s 288us/step - loss: 923.5147
Epoch 14/100
14400/14400 [==============================] - 5s 345us/step - loss: 826.2468
Epoch 15/100
14400/14400 [==============================] - 4s 287us/step - loss: 761.8801
Epoch 16/100
14400/14400 [==============================] - 4s 302us/step - loss: 673.4842
Epoch 17/100
14400/14400 [==============================] - 5s 329us/step - loss: 651.6575
Epoch 18/100
14400/14400 [==============================] - 5s 374us/step - loss: 600.7392
Epoch 19/100
14400/14400 [==============================] - 5s 319us/step - loss: 588.0273
Epoch 20/

14400/14400 [==============================] - 4s 283us/step - loss: 77.7429
Epoch 100/100
1600/1600 [==============================] - 1s 355us/step
Epoch 1/100
14400/14400 [==============================] - 5s 342us/step - loss: 2188.5565
Epoch 2/100
14400/14400 [==============================] - 5s 348us/step - loss: 1699.2489
Epoch 3/100
14400/14400 [==============================] - 4s 303us/step - loss: 1401.5686
Epoch 4/100
14400/14400 [==============================] - 4s 283us/step - loss: 1263.4891
Epoch 5/100
14400/14400 [==============================] - 4s 282us/step - loss: 1180.3346
Epoch 6/100
14400/14400 [==============================] - 5s 316us/step - loss: 1114.4536
Epoch 7/100
14400/14400 [==============================] - 4s 283us/step - loss: 1063.4065
Epoch 8/100
14400/14400 [==============================] - 4s 283us/step - loss: 991.9130
Epoch 9/100
14400/14400 [==============================] - 4s 282us/step - loss: 931.2768
Epoch 10/100
14400/14400 [=======

14400/14400 [==============================] - 4s 270us/step - loss: 173.7938
Epoch 89/100
14400/14400 [==============================] - 4s 270us/step - loss: 214.2240
Epoch 90/100
14400/14400 [==============================] - 4s 270us/step - loss: 159.4259
Epoch 91/100
14400/14400 [==============================] - 4s 255us/step - loss: 174.4881
Epoch 92/100
14400/14400 [==============================] - 4s 252us/step - loss: 163.0297
Epoch 93/100
14400/14400 [==============================] - 4s 286us/step - loss: 171.6680
Epoch 94/100
14400/14400 [==============================] - 4s 250us/step - loss: 158.8061
Epoch 95/100
14400/14400 [==============================] - 5s 357us/step - loss: 162.8162
Epoch 96/100
14400/14400 [==============================] - 4s 302us/step - loss: 169.4462
Epoch 97/100
14400/14400 [==============================] - 4s 300us/step - loss: 159.7950
Epoch 98/100
14400/14400 [==============================] - 4s 283us/step - loss: 156.9560
Epoch 99/100

14400/14400 [==============================] - 4s 294us/step - loss: 1132.4488
Epoch 77/100
14400/14400 [==============================] - 4s 261us/step - loss: 1144.5554
Epoch 78/100
14400/14400 [==============================] - 4s 257us/step - loss: 1134.4174
Epoch 79/100
14400/14400 [==============================] - 4s 310us/step - loss: 1133.6497
Epoch 80/100
14400/14400 [==============================] - 5s 323us/step - loss: 1137.3315
Epoch 81/100
14400/14400 [==============================] - 4s 286us/step - loss: 1131.1558
Epoch 82/100
14400/14400 [==============================] - 4s 284us/step - loss: 1132.1190
Epoch 83/100
14400/14400 [==============================] - 4s 283us/step - loss: 1135.6501
Epoch 84/100
14400/14400 [==============================] - 5s 322us/step - loss: 1129.4477
Epoch 85/100
14400/14400 [==============================] - 4s 285us/step - loss: 1139.5608
Epoch 86/100
14400/14400 [==============================] - 4s 286us/step - loss: 1138.6623
E

In [44]:
results

array([  -10.49144348,   -35.7423364 ,   -41.1019976 , -1719.53843256,
       -4603.63115074, -1982.68875484, -1328.72226267,   -64.94185337,
        -566.18876022, -5369.50226734])

In [55]:
# define model
def four_model():
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=8, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(64, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(32, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [56]:
estimator4 = KerasRegressor(build_fn=four_model, epochs=100, batch_size=5, verbose=1)

In [57]:
kfold = KFold(n_splits=5, random_state=seed)
results4 = cross_val_score(estimator4, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
12800/12800 [==============================] - 7s 554us/step - loss: 2358.03020s - los
Epoch 2/100
12800/12800 [==============================] - 5s 396us/step - loss: 2093.4634
Epoch 3/100
12800/12800 [==============================] - 5s 413us/step - loss: 2092.5004
Epoch 4/100
12800/12800 [==============================] - 6s 438us/step - loss: 1945.2697
Epoch 5/100
12800/12800 [==============================] - 5s 402us/step - loss: 1755.30030s - loss: 176
Epoch 6/100
12800/12800 [==============================] - 5s 404us/step - loss: 1739.5663
Epoch 7/100
12800/12800 [==============================] - 6s 443us/step - loss: 1727.2670
Epoch 8/100
12800/12800 [==============================] - 5s 409us/step - loss: 1666.0423
Epoch 9/100
12800/12800 [==============================] - 5s 406us/step - loss: 1503.2551
Epoch 10/100
12800/12800 [==============================] - 6s 447us/step - loss: 1463.6297
Epoch 11/100
12800/12800 [==============================] - 5s 418u

KeyboardInterrupt: 